pytorch == 2.3.0+cu118

python == 3.11.9

cuda == 11.8


In [1]:
import torch
from data_handler import DataHandler
from model import TravelDocClassifier
from cuda_utils import clear_cache, print_cuda_memory_info
from torch.utils.data import  Subset
import numpy as np

In [2]:
clear_cache()
print_cuda_memory_info()

Device: NVIDIA GeForce GTX 1660 Ti
Total Memory: 6.44 GB
Reserved Memory: 0.00 GB
Allocated Memory: 0.00 GB
Free Memory: 0.00 GB


In [3]:
from sklearn.model_selection import train_test_split

data_handler = DataHandler(tokenizer_name='bert-base-chinese')

encodings, labels = data_handler.load_encoded_data('encoded_data/train')

length = len(labels)

# 使用 train_test_split 生成訓練集和驗證集的索引
train_indices, val_indices = train_test_split(np.arange(length), test_size=0.2, random_state=42)

# 創建訓練集和驗證集的子集
train_dataset = Subset(data_handler.create_dataset(encodings, labels), train_indices)
val_dataset = Subset(data_handler.create_dataset(encodings, labels), val_indices)

c:\Users\a9542\Desktop\programes\information_retrieval\final\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

classifier = TravelDocClassifier(device=device)
stats = classifier.train(train_dataset, val_dataset, epochs=3, batch_size=16, learning_rate=5e-5, patience=2)

# 輸出結果
print(f'Best Val Loss: {stats["best_val_loss"]:.4f}, Best Val Accuracy: {stats["best_val_accuracy"]:.2f}%')
classifier.plot_metrics(stats)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
